In [1]:
import xarray as xr
import geopandas as gpd
import glob

In [2]:
ug = gpd.read_file(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/uganda_administrative_boundaries/simplified_uganda_districts.shp"
)

ug.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [3]:
ug.head()

,ADM2_EN,ADM1_EN,geometry
0,Abim,Northern,"POLYGON ((33.58099 3.14912, 33.58555 3.14702, ..."
1,Adjumani,Northern,"POLYGON ((32.04597 3.58896, 32.04578 3.58875, ..."
2,Agago,Northern,"POLYGON ((33.41652 3.30434, 33.41701 3.30403, ..."
3,Alebtong,Northern,"POLYGON ((33.03629 2.5013, 33.03664 2.50075, 3..."
4,Amolatar,Northern,"POLYGON ((32.9014 1.8057, 32.90231 1.80568, 32..."


In [4]:
ds = xr.open_dataset(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/totalcloudcover/totalcloudcover2007_2019.grib",
    engine="cfgrib",
)

ds

<xarray.Dataset> Size: 3GB
Dimensions:     (time: 113952, latitude: 24, longitude: 259)
Coordinates:
  * time        (time) datetime64[ns] 912kB 2007-01-01 ... 2019-12-31T23:00:00
  * latitude    (latitude) float64 192B 4.25 4.0 3.75 3.5 ... -1.0 -1.25 -1.5
  * longitude   (longitude) float64 2kB -29.5 -29.25 -29.0 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 912kB ...
Data variables:
    tcc         (time, latitude, longitude) float32 3GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T13:12 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
minx, miny, maxx, maxy = ug.total_bounds

ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
ds_ug

<xarray.Dataset> Size: 222MB
Dimensions:     (time: 113952, latitude: 22, longitude: 22)
Coordinates:
  * time        (time) datetime64[ns] 912kB 2007-01-01 ... 2019-12-31T23:00:00
  * latitude    (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude   (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 912kB ...
Data variables:
    tcc         (time, latitude, longitude) float32 221MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T13:12 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
ds_monthly = ds_ug.resample(time="1M").mean()
ds_monthly

c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 304kB
Dimensions:    (time: 156, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 1kB 2007-01-31 2007-02-28 ... 2019-12-31
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    tcc        (time, latitude, longitude) float32 302kB 0.507 0.5051 ... 0.7772
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T13:12 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
files = glob.glob(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/totalcloudcover/*.grib"
)

datasets = []

for file in files:
    ds = xr.open_dataset(file, engine="cfgrib")
    minx, miny, maxx, maxy = ug.total_bounds
    ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
    ds_monthly = ds_ug.resample(time="1M").mean()
    datasets.append(ds_monthly)

combined_ds = xr.concat(datasets, dim="time")

combined_ds

c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
Ignoring index file 'C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/totalcloudcover\\totalcloudcover2007_2019.grib.5b7b6.idx' incompatible with GRIB file
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 1MB
Dimensions:    (time: 539, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1994-01-31 1994-02-28 ... 1993-12-31
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    tcc        (time, latitude, longitude) float32 1MB 0.5204 0.5167 ... 0.547
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T13:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
df_cloud_cover = combined_ds.to_dataframe().reset_index()
df_cloud_cover

,time,latitude,longitude,tcc,number,step,surface
0,1994-01-31,4.00,29.75,0.520377,0,0 days,0.0
1,1994-01-31,4.00,30.00,0.516714,0,0 days,0.0
2,1994-01-31,4.00,30.25,0.504653,0,0 days,0.0
3,1994-01-31,4.00,30.50,0.493431,0,0 days,0.0
4,1994-01-31,4.00,30.75,0.477386,0,0 days,0.0
...,...,...,...,...,...,...,...
260871,1993-12-31,-1.25,34.00,0.638383,0,0 days,0.0
260872,1993-12-31,-1.25,34.25,0.625107,0,0 days,0.0
260873,1993-12-31,-1.25,34.50,0.609990,0,0 days,0.0
260874,1993-12-31,-1.25,34.75,0.583174,0,0 days,0.0


In [9]:
# convert grid cells points to shapely points
gdf = gpd.GeoDataFrame(
    df_cloud_cover,
    geometry=gpd.points_from_xy(df_cloud_cover.longitude, df_cloud_cover.latitude),
    crs="EPSG:4326",
)

In [10]:
gdf = gpd.sjoin(gdf, ug, how="left", predicate="within")
gdf

,time,latitude,longitude,tcc,number,step,surface,geometry,index_right,ADM2_EN,ADM1_EN
0,1994-01-31,4.00,29.75,0.520377,0,0 days,0.0,POINT (29.75 4),NaN,NaN,NaN
1,1994-01-31,4.00,30.00,0.516714,0,0 days,0.0,POINT (30 4),NaN,NaN,NaN
2,1994-01-31,4.00,30.25,0.504653,0,0 days,0.0,POINT (30.25 4),NaN,NaN,NaN
3,1994-01-31,4.00,30.50,0.493431,0,0 days,0.0,POINT (30.5 4),NaN,NaN,NaN
4,1994-01-31,4.00,30.75,0.477386,0,0 days,0.0,POINT (30.75 4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
260871,1993-12-31,-1.25,34.00,0.638383,0,0 days,0.0,POINT (34 -1.25),NaN,NaN,NaN
260872,1993-12-31,-1.25,34.25,0.625107,0,0 days,0.0,POINT (34.25 -1.25),NaN,NaN,NaN
260873,1993-12-31,-1.25,34.50,0.609990,0,0 days,0.0,POINT (34.5 -1.25),NaN,NaN,NaN
260874,1993-12-31,-1.25,34.75,0.583174,0,0 days,0.0,POINT (34.75 -1.25),NaN,NaN,NaN


In [11]:
df_monthly = gdf.groupby(["ADM2_EN", "ADM1_EN", "time"])["tcc"].mean().reset_index()
df_monthly

,ADM2_EN,ADM1_EN,time,tcc
0,Abim,Northern,1981-01-31,0.350660
1,Abim,Northern,1981-02-28,0.458239
2,Abim,Northern,1981-03-31,0.852498
3,Abim,Northern,1981-04-30,0.820665
4,Abim,Northern,1981-05-31,0.695249
...,...,...,...,...
64675,Zombo,Northern,2025-07-31,0.623064
64676,Zombo,Northern,2025-08-31,0.753444
64677,Zombo,Northern,2025-09-30,0.706700
64678,Zombo,Northern,2025-10-31,0.820268


In [12]:
# rename columns
df_monthly = df_monthly.rename(
    columns={
        "ADM2_EN": "District",
        "ADM1_EN": "Region",
        "time": "Date",
        "tcc": "Total_Cloud_Cover",
    }
)
df_monthly

,District,Region,Date,Total_Cloud_Cover
0,Abim,Northern,1981-01-31,0.350660
1,Abim,Northern,1981-02-28,0.458239
2,Abim,Northern,1981-03-31,0.852498
3,Abim,Northern,1981-04-30,0.820665
4,Abim,Northern,1981-05-31,0.695249
...,...,...,...,...
64675,Zombo,Northern,2025-07-31,0.623064
64676,Zombo,Northern,2025-08-31,0.753444
64677,Zombo,Northern,2025-09-30,0.706700
64678,Zombo,Northern,2025-10-31,0.820268


In [13]:
# save to csv
df_monthly.to_csv("uganda_monthly_total_cloud_cover_1981_2025.csv", index=False)